# Model Comparison

In [1]:
import pandas as pd
import torch
import torch.nn as nn
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score

In [2]:
LABELED_DATA = 'data/labelHistory.csv'
RESULTS_DIR = 'results'
MODELS_DIR = 'models'

In [3]:
MODEL_VERSIONS = [
    {'name': 'Model 1 (Baseline)', 'model': 'classifier.pth',   'vec': 'vectorizer.pkl'},
    {'name': 'Model 2 (Bugged)',   'model': 'classifier_2.pth', 'vec': 'vectorizer_2.pkl'},
    {'name': 'Model 3 (Final)',    'model': 'classifier_3.pth', 'vec': 'vectorizer_3.pkl'}
]

In [4]:
os.makedirs(RESULTS_DIR, exist_ok=True)

In [5]:
class WatchHistoryClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(WatchHistoryClassifier, self).__init__()

        self.main = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, num_classes),
        )
    def forward(self, x):
        return self.main(x)

In [6]:
def load_and_evaluate():
    # 1. Load Data
    print("📊 Loading Labeled Data...")
    df = pd.read_csv(LABELED_DATA)
    df['text'] = df['title'] + " " + df['channel'].fillna('')
    y_true = df['label'].values - 1
    
    classes = ['Learning', 'Entertainment', 'Music', 'Others']

    comparison_stats = []

    # 2. Iterate through models
    for version in MODEL_VERSIONS:
        print(f"\n🧪 Testing {version['name']}...")
        
        try:
            # Load Vectorizer
            vec_path = os.path.join(MODELS_DIR, version['vec'])
            vectorizer = joblib.load(vec_path)
            
            # Vectorize Data (Transform only)
            X = vectorizer.transform(df['text']).toarray()
            X_t = torch.FloatTensor(X)
            
            # Load Model
            model_path = os.path.join(MODELS_DIR, version['model'])
            input_dim = len(vectorizer.vocabulary_)
            
            # Initialize & Load Weights
            model = WatchHistoryClassifier(input_dim, num_classes=4)
            model.load_state_dict(torch.load(model_path))
            model.eval()
            
            # Predict
            with torch.no_grad():
                outputs = model(X_t)
                _, preds = torch.max(outputs, 1)
                y_pred = preds.numpy()

            # --- METRICS ---
            acc = accuracy_score(y_true, y_pred)
            f1 = f1_score(y_true, y_pred, average='weighted')
            
            print(f"   -> Accuracy: {acc:.4f}")
            print(f"   -> F1 Score: {f1:.4f}")
            
            comparison_stats.append({
                'Model': version['name'],
                'Accuracy': acc,
                'F1 Score': f1,
                'Precision': classification_report(y_true, y_pred, output_dict=True)['weighted avg']['precision'],
                'Recall': classification_report(y_true, y_pred, output_dict=True)['weighted avg']['recall']
            })

            # --- PLOT 1: CONFUSION MATRIX ---
            plt.figure(figsize=(8, 6))
            cm = confusion_matrix(y_true, y_pred)
            sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
            plt.title(f"Confusion Matrix: {version['name']}")
            plt.ylabel('Actual Category')
            plt.xlabel('Predicted Category')
            
            save_path = os.path.join(RESULTS_DIR, f"confusion_matrix_{version['model']}.png")
            plt.savefig(save_path)
            plt.close()
            print(f"   -> Saved Plot: {save_path}")

        except Exception as e:
            print(f"❌ Failed to process {version['name']}: {e}")
    if comparison_stats:
        stats_df = pd.DataFrame(comparison_stats)
        
        # Save raw stats to CSV
        csv_path = os.path.join(RESULTS_DIR, "model_comparison_metrics.csv")
        stats_df.to_csv(csv_path, index=False)
        print(f"\n💾 Saved comparison metrics to {csv_path}")

        # Melt for plotting
        melted_df = stats_df.melt(id_vars="Model", var_name="Metric", value_name="Score")
        
        plt.figure(figsize=(10, 6))
        sns.barplot(data=melted_df, x="Model", y="Score", hue="Metric", palette="viridis")
        plt.title("Model Performance Comparison")
        plt.ylim(0, 1.1)
        plt.legend(loc='lower right')
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        
        comp_plot_path = os.path.join(RESULTS_DIR, "model_comparison_chart.png")
        plt.savefig(comp_plot_path)
        plt.close()
        print(f"📊 Saved Comparison Chart to {comp_plot_path}")

In [7]:
load_and_evaluate()

📊 Loading Labeled Data...

🧪 Testing Model 1 (Baseline)...
❌ Failed to process Model 1 (Baseline): Error(s) in loading state_dict for WatchHistoryClassifier:
	Missing key(s) in state_dict: "main.0.weight", "main.0.bias", "main.3.weight", "main.3.bias". 
	Unexpected key(s) in state_dict: "fc1.weight", "fc1.bias", "fc2.weight", "fc2.bias". 

🧪 Testing Model 2 (Bugged)...
   -> Accuracy: 0.3345
   -> F1 Score: 0.3555
   -> Saved Plot: results/confusion_matrix_classifier_2.pth.png

🧪 Testing Model 3 (Final)...
   -> Accuracy: 0.9638
   -> F1 Score: 0.9639
   -> Saved Plot: results/confusion_matrix_classifier_3.pth.png

💾 Saved comparison metrics to results/model_comparison_metrics.csv
📊 Saved Comparison Chart to results/model_comparison_chart.png
